In [1]:
#This program uses an LSTM to predict the closing stock price of NVDA using the past 50 trading 
#day stock prices of NVDA and INTC
#Included stock price of INTC as its a similar stock
#Main objective was to learn how to create a Many to Many Model and understand how to
#input multiple timesteps and features
#I understand that the way I created my dataset isn't good as by just
#slicing 10 years of data into timesteps of 50, it only creates 55 samples which is insufficient
#Have also learned that creating the dataset is the most difficult part 

import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [13]:
df = pd.read_csv('NVDA2011to2022.csv')
#df = df.filter(items = ['NVDA', 'INTC'] ) # REMOVE DATE
X = df.iloc[:2750 , :]  #set training_dataset to multiple of 50
X = X.filter(items = ['NVDA', 'INTC'] ) # REMOVE DATE
X = np.array(X).reshape(55, 50, 2) #reshape into 55 samples, 50 time steps each, 2 features
X


array([[[  3.515   ,  23.9     ],
        [  3.455   ,  23.85    ],
        [  3.6625  ,  24.200001],
        ...,
        [  3.62    ,  25.59    ],
        [  3.5475  ,  25.799999],
        [  3.525   ,  25.75    ]],

       [[  3.4325  ,  25.139999],
        [  3.38    ,  25.040001],
        [  3.495   ,  25.389999],
        ...,
        [  3.61    ,  27.9     ],
        [  3.6375  ,  27.879999],
        [  3.68    ,  28.190001]],

       [[  3.705   ,  28.190001],
        [  3.79    ,  27.799999],
        [  3.8075  ,  28.16    ],
        ...,
        [  2.9325  ,  25.040001],
        [  3.0175  ,  25.43    ],
        [  3.0975  ,  26.07    ]],

       ...,

       [[237.139999,  47.93    ],
        [229.360001,  48.07    ],
        [213.520004,  47.68    ],
        ...,
        [175.949997,  44.009998],
        [166.300003,  42.830002],
        [161.75    ,  42.84    ]],

       [[177.059998,  43.599998],
        [172.639999,  43.080002],
        [181.770004,  44.400002],
        .

In [14]:
Y = [] #creating the prediction output which is the subsequent 10 trading days (2 weeks)
df_Y = df.filter(items = ['NVDA'] )
for i in range(50, 2760): #obtains the next 10 trading days after the first 50
    if i % 50 < 10:
        Y.append(df_Y[i+1:i+2])
Y = np.array(Y).reshape(55,10,1)
Y

array([[[  3.38    ],
        [  3.495   ],
        [  3.5875  ],
        [  3.555   ],
        [  3.6525  ],
        [  3.735   ],
        [  3.7125  ],
        [  3.6775  ],
        [  3.7275  ],
        [  3.7     ]],

       [[  3.79    ],
        [  3.8075  ],
        [  3.85    ],
        [  3.8325  ],
        [  3.77    ],
        [  3.6625  ],
        [  3.6575  ],
        [  3.61    ],
        [  3.54    ],
        [  3.585   ]],

       [[  3.03    ],
        [  3.065   ],
        [  3.1275  ],
        [  3.045   ],
        [  3.0075  ],
        [  3.0725  ],
        [  3.1     ],
        [  3.31    ],
        [  3.3625  ],
        [  3.21    ]],

       [[  3.6625  ],
        [  3.6525  ],
        [  3.66    ],
        [  3.5775  ],
        [  3.65    ],
        [  3.5875  ],
        [  3.5725  ],
        [  3.58    ],
        [  3.5125  ],
        [  3.5075  ]],

       [[  3.1375  ],
        [  3.1225  ],
        [  3.255   ],
        [  3.2525  ],
        [  3.1525  ],
  

In [18]:
#stacked LSTM model
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(50, 2)))
model.add(RepeatVector(10))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=500, validation_split=0.2, verbose=1, batch_size=10)


Epoch 1/500
5/5 [==============================] - 2s 78ms/step - loss: 2876.8857 - val_loss: 61860.3984
Epoch 2/500
5/5 [==============================] - 0s 19ms/step - loss: 2810.5588 - val_loss: 36587.7812
Epoch 3/500
5/5 [==============================] - 0s 20ms/step - loss: 28871.3066 - val_loss: 47852.0117
Epoch 4/500
5/5 [==============================] - 0s 19ms/step - loss: 5104.0889 - val_loss: 32636.4492
Epoch 5/500
5/5 [==============================] - 0s 19ms/step - loss: 3129.2048 - val_loss: 36599.3398
Epoch 6/500
5/5 [==============================] - 0s 19ms/step - loss: 5462.8335 - val_loss: 77026.6172
Epoch 7/500
5/5 [==============================] - 0s 19ms/step - loss: 1893.2859 - val_loss: 17585.1992
Epoch 8/500
5/5 [==============================] - 0s 19ms/step - loss: 2513.2468 - val_loss: 22873.8164
Epoch 9/500
5/5 [==============================] - 0s 19ms/step - loss: 1881.7793 - val_loss: 50431.3750
Epoch 10/500
5/5 [==============================] - 0s

5/5 [==============================] - 0s 19ms/step - loss: 207.1237 - val_loss: 26445.5391
Epoch 79/500
5/5 [==============================] - 0s 19ms/step - loss: 176.1926 - val_loss: 27408.0488
Epoch 80/500
5/5 [==============================] - 0s 19ms/step - loss: 183.8315 - val_loss: 26903.7188
Epoch 81/500
5/5 [==============================] - 0s 19ms/step - loss: 175.5394 - val_loss: 25591.9492
Epoch 82/500
5/5 [==============================] - 0s 19ms/step - loss: 191.4187 - val_loss: 25556.4004
Epoch 83/500
5/5 [==============================] - 0s 19ms/step - loss: 192.7843 - val_loss: 26506.1641
Epoch 84/500
5/5 [==============================] - 0s 19ms/step - loss: 171.2901 - val_loss: 26674.7754
Epoch 85/500
5/5 [==============================] - 0s 19ms/step - loss: 156.7158 - val_loss: 27274.2988
Epoch 86/500
5/5 [==============================] - 0s 19ms/step - loss: 164.7640 - val_loss: 27581.0254
Epoch 87/500
5/5 [==============================] - 0s 19ms/step - l

5/5 [==============================] - 0s 19ms/step - loss: 63.2527 - val_loss: 34060.9141
Epoch 157/500
5/5 [==============================] - 0s 19ms/step - loss: 63.2017 - val_loss: 34450.0234
Epoch 158/500
5/5 [==============================] - 0s 19ms/step - loss: 61.7881 - val_loss: 34343.6211
Epoch 159/500
5/5 [==============================] - 0s 19ms/step - loss: 61.1854 - val_loss: 34629.1172
Epoch 160/500
5/5 [==============================] - 0s 19ms/step - loss: 60.7925 - val_loss: 34625.1719
Epoch 161/500
5/5 [==============================] - 0s 19ms/step - loss: 60.7033 - val_loss: 34475.4141
Epoch 162/500
5/5 [==============================] - 0s 19ms/step - loss: 61.9885 - val_loss: 34697.0703
Epoch 163/500
5/5 [==============================] - 0s 19ms/step - loss: 59.3408 - val_loss: 34478.1641
Epoch 164/500
5/5 [==============================] - 0s 19ms/step - loss: 59.0687 - val_loss: 34228.4297
Epoch 165/500
5/5 [==============================] - 0s 20ms/step - l

5/5 [==============================] - 0s 19ms/step - loss: 47.9755 - val_loss: 37371.2656
Epoch 235/500
5/5 [==============================] - 0s 19ms/step - loss: 50.9192 - val_loss: 37259.3398
Epoch 236/500
5/5 [==============================] - 0s 19ms/step - loss: 48.0070 - val_loss: 37864.9258
Epoch 237/500
5/5 [==============================] - 0s 19ms/step - loss: 47.7252 - val_loss: 37745.1211
Epoch 238/500
5/5 [==============================] - 0s 19ms/step - loss: 44.1580 - val_loss: 37187.1250
Epoch 239/500
5/5 [==============================] - 0s 19ms/step - loss: 61.3115 - val_loss: 37256.9531
Epoch 240/500
5/5 [==============================] - 0s 19ms/step - loss: 42.6897 - val_loss: 38569.8438
Epoch 241/500
5/5 [==============================] - 0s 19ms/step - loss: 53.6464 - val_loss: 37854.5508
Epoch 242/500
5/5 [==============================] - 0s 19ms/step - loss: 47.6233 - val_loss: 36962.0586
Epoch 243/500
5/5 [==============================] - 0s 19ms/step - l

5/5 [==============================] - 0s 20ms/step - loss: 42.2838 - val_loss: 38009.7852
Epoch 313/500
5/5 [==============================] - 0s 20ms/step - loss: 42.3199 - val_loss: 38042.2109
Epoch 314/500
5/5 [==============================] - 0s 20ms/step - loss: 41.6152 - val_loss: 38142.5898
Epoch 315/500
5/5 [==============================] - 0s 20ms/step - loss: 41.0686 - val_loss: 37746.9414
Epoch 316/500
5/5 [==============================] - 0s 20ms/step - loss: 41.2587 - val_loss: 37851.2773
Epoch 317/500
5/5 [==============================] - 0s 20ms/step - loss: 40.1813 - val_loss: 38231.7227
Epoch 318/500
5/5 [==============================] - 0s 20ms/step - loss: 42.3364 - val_loss: 38336.7539
Epoch 319/500
5/5 [==============================] - 0s 20ms/step - loss: 45.7081 - val_loss: 38476.5430
Epoch 320/500
5/5 [==============================] - 0s 19ms/step - loss: 44.9487 - val_loss: 38008.3477
Epoch 321/500
5/5 [==============================] - 0s 21ms/step - l

5/5 [==============================] - 0s 19ms/step - loss: 40.0779 - val_loss: 44858.3945
Epoch 391/500
5/5 [==============================] - 0s 19ms/step - loss: 65.9165 - val_loss: 45667.4688
Epoch 392/500
5/5 [==============================] - 0s 19ms/step - loss: 63.7762 - val_loss: 46744.7227
Epoch 393/500
5/5 [==============================] - 0s 19ms/step - loss: 33.8569 - val_loss: 45578.7188
Epoch 394/500
5/5 [==============================] - 0s 19ms/step - loss: 51.2554 - val_loss: 46546.1680
Epoch 395/500
5/5 [==============================] - 0s 19ms/step - loss: 41.0487 - val_loss: 46760.7969
Epoch 396/500
5/5 [==============================] - 0s 19ms/step - loss: 44.3380 - val_loss: 46135.9258
Epoch 397/500
5/5 [==============================] - 0s 19ms/step - loss: 38.6973 - val_loss: 45812.1406
Epoch 398/500
5/5 [==============================] - 0s 19ms/step - loss: 39.0259 - val_loss: 45870.9141
Epoch 399/500
5/5 [==============================] - 0s 19ms/step - l

5/5 [==============================] - 0s 19ms/step - loss: 43.9834 - val_loss: 48017.3008
Epoch 469/500
5/5 [==============================] - 0s 19ms/step - loss: 36.8124 - val_loss: 47123.9727
Epoch 470/500
5/5 [==============================] - 0s 19ms/step - loss: 46.5893 - val_loss: 46962.6641
Epoch 471/500
5/5 [==============================] - 0s 19ms/step - loss: 35.4797 - val_loss: 47562.7852
Epoch 472/500
5/5 [==============================] - 0s 19ms/step - loss: 40.3256 - val_loss: 47650.3711
Epoch 473/500
5/5 [==============================] - 0s 19ms/step - loss: 37.7526 - val_loss: 47189.8867
Epoch 474/500
5/5 [==============================] - 0s 19ms/step - loss: 46.7563 - val_loss: 47514.9102
Epoch 475/500
5/5 [==============================] - 0s 19ms/step - loss: 36.8450 - val_loss: 47128.7422
Epoch 476/500
5/5 [==============================] - 0s 19ms/step - loss: 38.1733 - val_loss: 47487.0273
Epoch 477/500
5/5 [==============================] - 0s 19ms/step - l

In [19]:
#prediction for the next 10 days from 3 Nov
test_input = pd.read_csv('NVDATEST.csv')
test_input = test_input.filter(items = ['NVDA', 'INTC'] )
test_input = np.array(test_input).reshape(1,50,2)
test_input
test_output = model.predict(test_input, verbose=0)
print(test_output) #predicts a 

[[[82.207275]
  [80.95891 ]
  [71.27672 ]
  [72.897316]
  [75.4754  ]
  [76.47345 ]
  [74.70563 ]
  [72.67723 ]
  [70.040115]
  [66.75745 ]]]
